In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report


df = pd.read_csv('../DATASETS/L4/spam.csv', encoding='latin-1')
df = df[['Category', 'Message']]
df['Category_Label'] = df['Category'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(
    df['Message'], 
    df['Category_Label'], 
    test_size=0.3, 
    random_state=42
)

model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Звіт про класифікацію:")
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))

test_messages = [
    "Congratulations! You've won a $500 gift card. Click here to claim.",                                   # SPAM
    "Hey, are we still going to the gym today?",                                                            # HAM
    "Good morning, sweetheart. Do you wanna go to the cinema at the evening?",                              # HAM                
    "Dead on arrival? Not on our watch. Premium health coverage for those who can afford to live.",         # HAM
    "Give your brain the bandwidth it deserves. Neural processors starting at just 4999$.",                 # SPAM
    "Win Big! Become a Millionaire Overnight! The Night City Lottery is waiting for YOUR lucky numbers.",   # SPAM
    "Taste the Love! The only drink that hits your dopamine receptors directly. Buy one, get one FREE!"     # SPAM
]

results = model.predict(test_messages)
print("\n--- Результати для нових повідомлень ---")
for msg, res in zip(test_messages, results):
    label = "SPAM" if res == 1 else "HAM"
    print(f"[{label}] -> {msg}")

Звіт про класифікацію:
              precision    recall  f1-score   support

         Ham       0.99      1.00      0.99      1448
        Spam       0.98      0.95      0.97       224

    accuracy                           0.99      1672
   macro avg       0.99      0.97      0.98      1672
weighted avg       0.99      0.99      0.99      1672


--- Результати для нових повідомлень ---
[SPAM] -> Congratulations! You've won a $500 gift card. Click here to claim.
[HAM] -> Hey, are we still going to the gym today?
[HAM] -> Good morning, sweetheart. Do you wanna go to the cinema at the evening?
[HAM] -> Dead on arrival? Not on our watch. Premium health coverage for those who can afford to live.
[HAM] -> Give your brain the bandwidth it deserves. Neural processors starting at just 4999$.
[SPAM] -> Win Big! Become a Millionaire Overnight! The Night City Lottery is waiting for YOUR lucky numbers.
[HAM] -> Taste the Love! The only drink that hits your dopamine receptors directly. Buy one, g

# Висновки

Реалізований спам-фільтр продемонстрував високу точність на рівні 99%, що підтверджує ефективність наївного байєсівського класифікатора для обробки природної мови. Модель успішно ідентифікувала класичні спам-повідомлення про виграші, проте аналіз результатів у файлі виявив цікаві випадки: повідомлення зі специфічною лексикою ("dopamine receptors", "neural processors") були класифіковані як "HAM", хоча за змістом мали рекламний характер. Це пояснюється тим, що класифікатор базується на апріорній ймовірності класів та ймовірності вектора ознак, які розраховуються на основі частоти слів у навчальній вибірці

Проблема нульової ймовірності виникає у випадках, коли в аналізованому повідомленні зустрічається слово, якого не було в навчальній вибірці для певного класу. Оскільки ймовірність правила розраховується як добуток ймовірностей для кожної окремої змінної, наявність хоча б одного нульового значення автоматично занулює загальний результат для всього класу. Для розв'язання цієї проблеми застосовується методика оціночної функції Лапласа, яка передбачає додавання певного ненульового значення до кожної ймовірності. Це дозволяє уникнути математичних помилок при зустрічі з новими даними та забезпечує стабільність роботи класифікатора.